In [ ]:
import time
#from time import sleep
from pop import Pilot
#from pop.Pilot import AutoCar

car = Pilot.AutoCar()   # 객체 생성

# 차량 제어 - 조향
car.steering = -1   # 좌
time.sleep(1)
car.steering = 1    # 우

# 차량 제어 - 전진, 후진
car.forward()   # 전진
time.sleep(1)
car.backward()  # 후진
time.sleep(1)
car.stop()

# 차량 제어 - 속도 제어
car.setSpeed(50)
car.forward()
time.sleep(1)
car.backward(99)
time.sleep(1)
car.stop()

In [ ]:
# 차량 제어 - 가상 조이스틱
car.joystick()

In [ ]:
# 카메라 제어 - 좌,우,위,아래
car.camPan(0)
time.sleep(1)
car.camPan(180)
time.sleep(1)
car.camTilt(180)
time.sleep(1)
car.camTilt(0)

In [ ]:
# 6축 센서
value = car.getGyro()
print(value)

time.sleep(1)

value = car.getGyro('z')
print(value)
value = car.getGyro('y')
print(value)
value = car.getGyro('x')
print(value)

In [ ]:
# LiDAR 센서
from pop.Util import imshow
from pop import LiDAR
lidar = LiDAR.Rplidar()

lidar.connect()
lidar.startMotor()

vectors = lidar.getVectors()
for v in vectors:
    print(v)

coords = lidar.getXY()
for c in coords:
    print(c)

lidar_map = lidar.getMap(limit_distance=2000,size=(300,300))
#lidar_map = lidar.getMap(limit_distance=2000,size=(300,300))
imshow("map", lidar_map)

lidar.stopMotor()



In [ ]:
# LiDAR 응용 제어
from pop import LiDAR, Pilot

lidar = LiDAR.Rplidar()
car = Pilot.AutoCar()

lidar.connect()
lidar.startMotor()

# 주행속도를 99로 차량의 조향을 가운데로 설정
car.setSpeed(99)
car.steering = 0

# LiDAR에서 벡터값을 받아 전방 120도 범위 내에 물체를 감지
while True:
    no_collision = True

    vectors = lidar.getVectors()
    for v in vectors:
        degree = v[0]
        distance = v[1]

        if degree <= 60 or degree >= 300:
            if distance <= 700:
                no_collision = False
        
        if no_collision:
            car.setSpeed(99)
            car.forward()
        else:
            if car.steering == 0:
                car.stop()
            car.steering = -1
            car.backward(50)

In [ ]:
car.stop()
lidar.stopMotor()

In [ ]:
# 카메라 캡쳐
import cv2
from pop import Util

Util.enable_imshow()

cam = Util.gstrmer(width=640,height=480)
camera = cv2.VideoCapture(cam, cv2.CAP_GSTREAMER)
if not camera.isOpened():
    print("Not found camera")

width = camera.get(cv2.CAP_PROP_FRAME_WIDTH)
height = camera.get(cv2.CAP_PROP_FRAME_HEIGHT)
print("init width:%d, init height:%d" %(width,height))

for _ in range(120):
    ret, frame = camera.read()
    if not ret:
        break

    cv2.imshow("soda", frame)

camera.release()
cv2.destroyAllWindows()

In [ ]:
# 블로킹 모드 재생
import pyaudio
import wave

w = wave.open("/usr/share/sounds/alsa/Side_Left.wav","rb")
data = w.readframes(w.getnframes())
w.close()

p = pyaudio.PyAudio()
stream = p.open(format=p.get_format_from_width(2),
                channels=1,
                rate=48000,
                output=True)

data = w.readframes(1024)
while len(data) > 0:
    stream.write(data)
    data = w.readframes(1024)

w.close()
stream.stop_stream()
stream.close()
p.terminate()

In [ ]:
# 논블로킹 모드 재생
import pyaudio
import wave
import time

def callback(in_data,frame_count,time_info,status):
    data = w.readframes(frame_count)
    return(data,pyaudio.paContinue)

w = wave.open("/usr/share/sounds/alsa/Side_Left.wav","rb")
p = pyaudio.PyAudio()

stream = p.open(format=p.get_format_from_width(w.getsampwidth()),
                channels=w.getnchannels(),
                rate=w.getframerate(),
                output=True,
                stream_callback=callback)

stream.start_stream()

while stream.is_active():
    print("main work...")
    time.sleep(0.1)

stream.stop_stream()
stream.close()
p.terminate()

In [ ]:
# 블로킹 모드 녹음 5초
import pyaudio
import wave

CHUNK = 1024
RATE = 48000

p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paInt16, channels=1, rate=RATE,
                input=True, frames_per_buffer=CHUNK)
w = wave.open("./out.wav", "wb")
w.setnchannels(1)
w.setsampwidth(p.get_sample_size(pyaudio.paInt16))
w.setframerate(RATE)

TIME = 5 # 5초
data = []

for _ in range(0, int(RATE / CHUNK * TIME)):
    d = stream.read(CHUNK)
    data.append(d)

w.writeframes(b''.join(data))

w.close()
stream.stop_stream()
stream.close()
p.terminate()